Computation of forward model for scatterers in a block in immersion.

Nicolas Budyn, 2017


Input
-----

conf.yaml

Output
------

- Figures if ``aplt.conf['savefig'] == True``
- ``tfm_intensities.csv`` is ``save == True``

Model
-----

The response of a scatterer in $y$ for a given mode, for the transmitter $i$ and the receiver $j$ is:

$$
\begin{align}
F_{ij}(\omega) &= Q_i(\omega, y) Q'_j(\omega, y) S_{ij}(\omega, y) 
                  \exp(\iota \omega \tau_{ij}(y))  U(\omega) \\
&= H_{ij}(\omega) U(\omega)
\end{align}
$$
                  
                  
                  
$U(\omega)$ is the toneburst: variables ``toneburst`` in time domain and ``toneburst_f`` in frequency domain.

The scatterering function $S_{ij}$ is defined in variable ``scat_obj``.

The transfer function $H_{ij}$ is stored in variable ``transfer_function_f``.

Data structure
--------------

``frame`` : Frame
    Frame that contains the response of the scatterers for all views.
    
``tfms`` : dict of arim.im.tfm.TfmResult
    TFM image of the contribution of a mode to the corresponding view.
    



In [ ]:
matplotlib notebook

In [ ]:
import math
import logging
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml
from scipy.signal import hilbert

import arim, arim.model, arim.scat, arim.plot as aplt
import arim.models.block_in_immersion as bim
import arim.im, arim.signal # for imaging
import arim.scat

In [ ]:
save = True
aplt.conf['savefig'] = False

use_multifreq = True
max_number_of_reflection = 1
tfm_unique_only = False
numangles_for_scat_precomp = 120 # 0 to disable precomputation

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.getLogger('arim').setLevel(logging.INFO)

with open('conf.yaml', 'rb') as f:
    conf = arim.config.Config(yaml.load(f))
conf

#### Define inspection set-up

In [ ]:
# Make probe
probe = arim.Probe.make_matrix_probe(**conf['probe'])

probe.set_reference_element(conf['probe_location']['ref_element'])
probe.reset_position()
probe.rotate(arim.geometry.rotation_matrix_y(
        np.deg2rad(conf['probe_location']['angle_deg'])))
probe.translate([0, 0, conf['probe_location']['standoff']])

tx_list, rx_list = arim.ut.fmc(probe.numelements)
numscanlines = len(tx_list)

# Set up geometry and materials

couplant = arim.Material(**conf['couplant_material'])
block = arim.Material(**conf['block_material'])
frontwall = \
    arim.geometry.points_1d_wall_z(**conf['frontwall'], name='Frontwall')
backwall = \
    arim.geometry.points_1d_wall_z(**conf['backwall'], name='Backwall')
examination_object = arim.BlockInImmersion(block, couplant, frontwall, backwall)


scatterers = arim.geometry.default_oriented_points(
        arim.Points([(s['x'], 0., s['z']) for s in conf['scatterers']],
                     'Scatterer(s)'))
numscatterers = len(scatterers.points)

# for imaging:
grid = arim.geometry.Grid(**conf['grid'], ymin=0., ymax=0.)

aplt.plot_interfaces([probe.to_oriented_points(), frontwall, backwall,
                      grid.to_oriented_points(), scatterers],
                      show_last=True, markers=['.', '-', '-', 'k.', 'd'])

#### View definition and ray tracing

In [ ]:
# Ray tracing
views = bim.make_views(examination_object, probe.to_oriented_points(), scatterers,
                       max_number_of_reflection, tfm_unique_only)
# views = {'L-L': views['L-L']}  # debug
print('Views: ' + ', '.join(views.keys()))
arim.ray.ray_tracing(views.values(), convert_to_fortran_order=True)

# for imaging:
views_imaging = bim.make_views(examination_object, probe.to_oriented_points(),
                       grid.to_oriented_points(), max_number_of_reflection, tfm_unique_only)
# views_imaging = {'L-L': views_imaging['L-L']}  # debug
arim.ray.ray_tracing(views_imaging.values(), convert_to_fortran_order=True)

#### Toneburst and time vector

In [ ]:
max_delay = max((view.tx_path.rays.times.max() + view.rx_path.rays.times.max()
    for view in views.values()))
    
dt = .25 / conf['probe']['frequency'] # to adjust so that the whole toneburst is sampled
_tmax = 2 * max_delay + conf['toneburst']['num_cycles'] / conf['probe']['frequency']

numsamples = math.ceil(_tmax / dt)
time = arim.Time(0., dt, numsamples)
freq_array = np.fft.rfftfreq(len(time), dt)
numfreq = len(freq_array)

toneburst = arim.model.make_toneburst(conf['toneburst']['num_cycles'],
                                      conf['probe']['frequency'], dt,
                                      numsamples, wrap=True)
toneburst_f = np.fft.rfft(toneburst)

toneburst_ref = np.abs(hilbert(toneburst)[0])

# plot toneburst
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(1e6 * time.samples, toneburst)
plt.title('toneburst (time domain)')
plt.xlabel('time (µs)')

plt.subplot(1, 2, 2)
plt.plot(1e-6 * np.fft.rfftfreq(len(toneburst), dt),
         abs(toneburst_f))
plt.title('toneburst (frequency domain)')
plt.xlabel('frequency (MHz)')
if aplt.conf['savefig']:
    plt.savefig('toneburst')


#### Compute transfer function

In [ ]:
scat_obj = arim.scat.scat_factory(material=examination_object.block_material,
                                  **conf['scatterer'])
scat_angle = np.deg2rad(conf['scatterer_angle_deg'])

transfer_function_f = np.zeros((numscanlines, numfreq), np.complex_)
tfms = OrderedDict()

if use_multifreq:
    # Multi frequency model
    transfer_function_iterator = bim.multifreq_scat_transfer_functions(
        views, tx_list, rx_list, freq_array=freq_array,
        scat_obj=scat_obj, scat_angle=conf['scatterer_angle_deg'],
        probe_element_width=probe.dimensions.x[0],
        numangles_for_scat_precomp=numangles_for_scat_precomp)
else:
    # Single frequency model
    transfer_function_iterator = bim.singlefreq_scat_transfer_functions(
        views, tx_list, rx_list, freq_array=freq_array,
        scat_obj=scat_obj, scat_angle=conf['scatterer_angle_deg'],
        frequency=probe.frequency,
        probe_element_width=probe.dimensions.x[0],
        numangles_for_scat_precomp=numangles_for_scat_precomp)

with arim.helpers.timeit('Main loop'):
    for viewname, partial_transfer_func in transfer_function_iterator:
        transfer_function_f += partial_transfer_func

        # imaging:
        partial_response = arim.signal.rfft_to_hilbert(partial_transfer_func * toneburst_f,
                                                       numsamples)
        partial_frame = arim.Frame(partial_response, time, tx_list, rx_list,
                                   probe, examination_object)

        tfms[viewname] = arim.im.tfm.tfm_for_view(partial_frame, grid,
                    views_imaging[viewname],
                    interpolation='linear', fillvalue=0.)

#### Compute the response in frequency then time domain

In [ ]:
response_scanlines_f = transfer_function_f * toneburst_f
response_scanlines = np.fft.irfft(response_scanlines_f, numsamples, axis=-1)

frame = arim.Frame(response_scanlines, time, tx_list, rx_list,
                   probe, examination_object)

plt.figure()
idx = 0
plt.plot(frame.time.samples * 1e6, frame.scanlines[idx], label=f'tx={frame.tx[idx]}, rx={frame.tx[idx]}')
plt.xlabel('time (µs)')
plt.title('time-domain response')
plt.legend()
if aplt.conf['savefig']:
    plt.savefig('time_domain_response')


#### Check reciprocity

In [ ]:
tx = 0
rx = 1

idx1 = np.nonzero(np.logical_and(tx_list == tx, rx_list == rx))[0][0]
idx2 = np.nonzero(np.logical_and(tx_list == rx, rx_list == tx))[0][0]

plt.figure()
plt.plot(time.samples * 1e6, response_scanlines[idx1], label=f'tx={tx_list[idx1]}, rx={rx_list[idx1]}')
plt.plot(time.samples * 1e6, response_scanlines[idx2], label=f'tx={tx_list[idx2]}, rx={rx_list[idx2]}')
plt.legend()
plt.xlabel('time (µs)')
plt.title('reciprocity - signals must overlap perfectly')
if aplt.conf['savefig']:
    plt.savefig('reciprocity')

response_scanlines_1 = response_scanlines.reshape((probe.numelements, probe.numelements, len(time)))
response_scanlines_2 = np.swapaxes(response_scanlines_1, 0, 1)
error_reciprocity = np.max(np.abs(response_scanlines_1 - response_scanlines_2))
logger.info(f'Reciprocity error: {error_reciprocity}')

#### Measure TFM intensities

In [ ]:
tmp = []
scatterer_idx = grid.closest_point(*scatterers.points[0])

for viewname, tfm in tfms.items():
    max_tfm_idx = np.argmax(np.abs(tfm.res))
    tmp.append((
        viewname,
        arim.ut.decibel(np.abs(tfm.res.flat[scatterer_idx]), toneburst_ref),
        arim.ut.decibel(np.abs(tfm.res.flat[max_tfm_idx]), toneburst_ref),
        grid.x.flat[max_tfm_idx],
        grid.y.flat[max_tfm_idx],
        grid.z.flat[max_tfm_idx],
    ))
intensities_df = pd.DataFrame(tmp, columns=[
        'view', 'intensity_at_centre', 'max_intensity',
        'x_max_intensity', 'y_max_intensity', 'z_max_intensity']).set_index('view')

if save:
    intensities_df.to_csv('tfm_intensities.csv')
    
intensities_df.iloc[:, :2]

#### Plot TFM

In [ ]:
scale = aplt.common_dynamic_db_scale([tfm.res for tfm in tfms.values()])

ncols = 3
nrows = math.ceil(len(tfms) / ncols)
fig, axes = plt.subplots(nrows, ncols, figsize=(10, 30), sharex=True, sharey=True)

for i, (viewname, tfm) in enumerate(tfms.items()):
    ref_db, clim = next(scale)
    
    amp = intensities_df.loc[viewname]
    
    ax = axes.flat[i]
    plt.sca(ax)
    
    aplt.plot_tfm(tfm, ax=ax, clim=clim, scale='db', ref_db=ref_db,
                  title=f'TFM {viewname}',
                  interpolation='none', savefig=False)
    
    # last row
    if i // ncols == nrows - 1:
        plt.xlabel('x (mm)')
    else:
        plt.xlabel('')
    # first column
    if i % ncols == 0:
        plt.ylabel('z (mm)')
    else:
        plt.ylabel('')
        
    plt.plot(amp['x_max_intensity'], amp['z_max_intensity'], '1m')
    plt.plot(scatterers.points.x, scatterers.points.z, 'dm')

if aplt.conf['savefig']:
    ax.figure.savefig(f'tfm')